In [2]:
import pandas as pd
import os 
import re
import pickle
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from datetime import datetime
import numpy as np

os.chdir(r'C:\\Users\\PC\\Desktop\\pricing_project\\data')
os.listdir()

custom = pd.read_csv(os.listdir()[0])
master = pd.read_csv(os.listdir()[1])
product = pd.read_csv(os.listdir()[2])
search1 = pd.read_csv(os.listdir()[3])   
search2 = pd.read_csv(os.listdir()[-2])
session = pd.read_csv(os.listdir()[-1])


C:\Users\PC\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#%% 1. Data preprocessing

#%% (1) Dataset 'Product'

# -> 3개의 키값에서 2개의 키값으로 agg

# 구매가격 변수를 str -> int 변환.
product['PD_BUY_AM'] = list(map(lambda x:x.replace(",",""),product['PD_BUY_AM']))
product['PD_BUY_AM'] = product['PD_BUY_AM'].astype(int)

# 구매개수 변수를 str&int -> int로 변환.
product['PD_BUY_CT'] = product['PD_BUY_CT'].astype(str)
product['PD_BUY_CT'] = list(map(lambda x:x.replace(",",""),product['PD_BUY_CT']))
product['PD_BUY_CT'] = product['PD_BUY_CT'].astype(int)

## product에 새로운 열 "TOT_AM" 생성 (PD_BUY_AM는 제품 하나 당 개수이므로, 이를 구매한 제품의 갯수와 곱한 "총 지출 금액"이 "TOT_AM"임)
product["TOT_AM"] = product["PD_BUY_AM"] * product["PD_BUY_CT"]

# CLNT_ID와 SESS_ID가 모두 같은 행들을 "TOT_AM","PD_BUY_CT","PD_BUY_AM"에 대해 합계,평균,표준편차를 구한 것
product_agg = product.groupby(['CLNT_ID', 'SESS_ID'])[['TOT_AM','PD_BUY_CT','PD_BUY_AM']].agg(['sum','mean'])
product_agg.columns= list(map(lambda x:x[0]+'_'+x[1],list(product_agg)))



In [4]:

#%% (2) Dataset 'Session'

# SESS_DT을 datetime 자료형으로 변환.
session['SESS_DT'] = pd.to_datetime(session['SESS_DT'], format = '%Y%m%d')
## 월,주,일 변수 생성. 19 -> 1월 1일 이후 19번째 주 double check  0 = 월요일, 6 = 일요일  double check
session['MONTH'] = list(map(lambda x:x.month,session['SESS_DT'])) 
session['WEEK'] = list(map(lambda x:x.week,session['SESS_DT'])) 
session['DAY'] = list(map(lambda x:x.weekday(),session['SESS_DT'])) 


In [5]:
#%% (3) Dataset 'search1','search2'

# 서로 다른 key구조를 모델에 적용가능한 형태로 통일.

# merge를 위해 SESS_DT 형식 동일하게 변경. 
search2['SESS_DT'] = pd.to_datetime(search2['SESS_DT'], format = '%Y%m%d')

# 검색량 변수를 str&int -> int로 변환 후 이름 변경.
search2['SEARCH_CNT'] = search2['SEARCH_CNT'].astype(str)
search2['SEARCH_CNT'] = list(map(lambda x:x.replace(",",""), search2['SEARCH_CNT']))
search2['SEARCH_CNT'] =  search2['SEARCH_CNT'].astype(int)
search2.rename(columns={'SEARCH_CNT': 'SEARCH_TOT'}, inplace=True) # Search1과 컬럼명이 동일하지만 의미가 다르므로 이름 변경.

# 전체검색량, 검색 키워드 갯수, 개인검색량, 전체검색량 대비 개인 검색량, 변수 생성.
search = pd.merge(search1,session.loc[:,['CLNT_ID','SESS_ID','SESS_DT']],how = 'left', on = ['CLNT_ID','SESS_ID']) 
search = pd.merge(search,search2.loc[:,['SESS_DT','KWD_NM','SEARCH_TOT']],how = 'left', on = ['KWD_NM','SESS_DT']) 
cnt = search.groupby(['CLNT_ID','SESS_ID']).count()['KWD_NM'] # 순서 유의.
search = search.groupby(['CLNT_ID','SESS_ID']).sum() # 이 부분에서 고유한 키값으로 줄어듬. 
search['KWD_CNT'] = cnt
search['SEARCH_RATIO'] = search.SEARCH_CNT / search.SEARCH_TOT 

In [6]:
#%% (4) Make y 

## y labeling
# 날짜 차이 구하기
session = session.sort_values(['CLNT_ID','SESS_DT']) # diff를 사용하기 위해 날짜순으로 정렬
session['DT_DIFF'] = session['SESS_DT'].diff() # (1) 일단은 전체에 대해 차이를 구해준 다음
session.loc[session.CLNT_ID != session.CLNT_ID.shift(),'DT_DIFF'] = None #(2) CLNT_ID가 변하는 경우에만 None로 수정
# 새롭게 라벨 제작
Y = session['DT_DIFF'].dt.days.tolist() # date 형태를 int 형태로 변형
a = list()
a.append(np.nan)
Y = Y[1:]+a # 한칸씩 올리고 마지막에 np.nan 추가 
session['y']=Y #라벨 추가
session = session[pd.notnull(session['y'])] 

In [ ]:
#%% (5) Product Vector mapping

# #### SESS_ID마다 구매한 상품 쌓아 - 그 대분류 쌓아 - 대분류 구매 패턴 (빈도 / 여부)
# #### 변수 1 : 세션 내 쇼핑 Category 구매 빈도(단순 횟수)
# #### 변수 2 : 세션 내 쇼핑 Category 구매 여부(0,1 binary vec)

product = product.sort_values(by=['CLNT_ID', 'SESS_ID'], axis=0, ascending=[True, False])
master= master.sort_values(by='PD_C',ascending=True)
product_dummy =product.merge(master,on='PD_C',how='left')

#사전식으로 대분류 배열 정렬 (ㄱ으로 시작하여 ㅎ으로 끝나도록)
clac1_list=list(product_dummy['CLAC1_NM'].unique())
clac1_list.sort()
CLAC1_NM_dict=dict(zip(clac1_list,range(0,37)))

#대분류 한글 -> 배정된 숫자로 변경
product_dummy=product_dummy.replace({"CLAC1_NM": CLAC1_NM_dict})
#대분류 더미변수화
product_dummy = pd.concat([product_dummy, pd.get_dummies(product_dummy['CLAC1_NM'])], axis=1)
#세션별 대분류 더미변수 갯수 
product_dummy_agg= product_dummy.groupby(['CLNT_ID', 'SESS_ID'])[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36].agg(['sum'])
#groupby -> dataframe 
product_dummy_agg=pd.DataFrame(product_dummy_agg)
product_dummy_agg = product_dummy_agg.reset_index()



In [7]:
#%% (6) Merge datasets

raw = pd.merge(session,custom, how = 'left', on = ['CLNT_ID']) 
raw = pd.merge(raw,product_agg, how = 'left', on = ['CLNT_ID','SESS_ID']) 
raw = pd.merge(raw,search,how = 'left', on = ['CLNT_ID','SESS_ID']) 
raw = pd.merge(raw,product_dummy_agg, how = 'left', on = ['CLNT_ID','SESS_ID']) 
raw = raw.reindex(columns=sorted(list(raw)))
    

In [139]:
#%% (7) preprocessing of of raw data for deep learning

# age selection

raw['CLNT_AGE'].fillna(0, inplace=True) # 비회원 0
raw = raw[raw['CLNT_AGE']<50] # 비회원 + 20 ~40 대 

# dummy

## CLNT_AGE : float to object
raw['CLNT_AGE'] = raw['CLNT_AGE'].astype(object)

## 도시명은 163개 Unique 값으로, dummy가 너무 많아져 삭제

## 성별(CLNT_GENDER), 사용자 기기(DVC_CTG_NM), 행정구역(ZON_NM),나이대(CLNT_AGE) encoding
raw=pd.concat([raw, pd.get_dummies(raw[['CLNT_GENDER','ZON_NM','DVC_CTG_NM','CLNT_AGE']])], axis =1 )

# Deletion  ; 기존 string 변수, 모델링에 쓰지 않을 변수 삭제
del(raw2['DT_DIFF'])
del(raw2['CLNT_AGE'])
del(raw2['SESS_DT'])
del(raw2['CITY_NM'])
del(raw2['CLNT_GENDER'])
del(raw2['DVC_CTG_NM'])
del(raw2['ZON_NM'])
del(raw2['CLNT_ID'])
del(raw2['SESS_ID'])

# column order setting
cols = raw.columns.tolist()
cols = cols[:16] + cols[17:-1] + [cols[16]]
raw = raw[cols]

# data type change
for i in cols : 
    raw[i] = pd.to_numeric(raw[i], errors='coerce').fillna(0, downcast='infer')
raw = raw[cols].astype(float)


In [152]:
raw.dtypes

DAY                         float64
KWD_CNT                     float64
MONTH                       float64
PD_BUY_AM_mean              float64
PD_BUY_AM_sum               float64
PD_BUY_CT_mean              float64
PD_BUY_CT_sum               float64
SEARCH_CNT                  float64
SEARCH_RATIO                float64
SEARCH_TOT                  float64
SESS_SEQ                    float64
TOT_AM_mean                 float64
TOT_AM_sum                  float64
TOT_PAG_VIEW_CT             float64
TOT_SESS_HR_V               float64
WEEK                        float64
(0, sum)                    float64
(1, sum)                    float64
(2, sum)                    float64
(3, sum)                    float64
(4, sum)                    float64
(5, sum)                    float64
(6, sum)                    float64
(7, sum)                    float64
(8, sum)                    float64
(9, sum)                    float64
(10, sum)                   float64
(11, sum)                   

In [153]:
with open('raw.pickle','wb')as handle:
    pickle.dump(raw,handle)

In [ ]:
# Supplimentary EDA


#날짜 차이를 기준으로 그룹형성 -> 갯수(size) 구하기 -> index제거
test=raw.groupby('DT_DIFF',as_index=False).size().reset_index(name = 'count')


import matplotlib.pyplot as plt 
plt.plot(test['DT_DIFF'],test['count'])


def findSplitPoint(test, n) : 
   
    left = list()
    right = list()
    splitpoint = list()
    
    leftSum = 0 
    # traverse array element 
    for i in range(0, n) : 
       
        # add current element to left Sum 
        leftSum += test.iloc[i,1]  
        left.append(leftSum)
        splitpoint.append(i+1)
        # find sum of rest array elements (rightSum) 
        rightSum = 0 
        for j in range(i+1, n) : 
            rightSum += test.iloc[j,1]
        
        right.append(rightSum)
    # split poindex 
    return left,right,splitpoint


left,right,splitpoint = findSplitPoint(test,180)

Decider =pd.DataFrame(np.vstack([splitpoint,left, right]).T,
              columns=['splitpoint','leftsum', 'rightsum'])